<h1> Preprocessing

In [78]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import re

import sys
import os
sys.path.append('../../..')
sys.path.append('..')
from ReSt.src.data.preprocessing import Preprocessing

%load_ext autoreload
%autoreload 2

Already downloaded a model for the 'it' language
Reading english - 1grams ...
842
3303
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
# names of files to read from
train_val_AB_TSV = '../../../ReSt/dataset/haspeede2/raw/haspeede2_dev/haspeede2_dev_taskAB.tsv'

test_tweets_AB_TSV = '../../../ReSt/dataset/haspeede2/raw/haspeede2_test/haspeede2_test_taskAB-tweets.tsv'
test_news_AB_TSV = '../../../ReSt/dataset/haspeede2/raw/haspeede2_test/haspeede2-test_taskAB-news.tsv'

reference_tweets_AB_TSV = '../../../ReSt/dataset/haspeede2/raw/haspeede2_reference/haspeede2_reference_taskAB-tweets.tsv'
reference_news_AB_TSV = '../../../ReSt/dataset/haspeede2/raw/haspeede2_reference/haspeede2_reference_taskAB-news.tsv'

italian_words = '../../../ReSt/dataset/words/parole_uniche.txt'
italian_gzip = '../../../ReSt/dataset/words/italian_words.txt.gz'
bad_words = '../../../ReSt/dataset/words/lista_badwords.txt'
word_polarity_xml = '../../../ReSt/dataset/it-sentiment_lexicon.lmf.xml'

In [80]:
italian_dict = [word.rstrip().lower() for word in open(italian_words, 'r', encoding='utf8') if word.rstrip().lower() != '']
bad_words_dict = [word.rstrip().lower() for word in open(bad_words, 'r', encoding='utf8') if word.rstrip().lower() != '']

In [81]:
#Word polarity dict

# Reading the data inside the xml 
# file to a variable under the name  
# data 
with open(word_polarity_xml, 'r') as f: 
    data = f.read() 

# Passing the stored data inside 
# the beautifulsoup parser, storing 
# the returned object  
Bs_data = BeautifulSoup(data, "xml") 

word_polarity = {}

lemma_unique = Bs_data.find_all('Lemma')         #Finding all instances of tag 'Lemma'
sentiment_unique = Bs_data.find_all('Sentiment') 

if len(lemma_unique) != len(sentiment_unique):
    print('ERRORE')

for i in range(0, len(lemma_unique)):
    word = lemma_unique[i].get('writtenForm') #Extracting the data stored in a specific attributes of the 'Lemma' tag
    word = re.sub(r'_', ' ', word)
    
    polarity = sentiment_unique[i].get('polarity')
    if polarity is None:
        polarity = 'neutral'
    
    word_polarity[word] = polarity
inv_map = {v: k for k, v in word_polarity.items()}
inv_map.keys()

dict_keys(['negative', 'neutral', 'positive', 'nneutral'])

In [82]:
print(inv_map['nneutral'])

aspettare


In [83]:
# fix word polarity typo
word_polarity['aspettare'] = 'neutral'
inv_map = {v: k for k, v in word_polarity.items()}
inv_map.keys()

dict_keys(['negative', 'neutral', 'positive'])

## load dev set

In [85]:
#df = pd.read_csv(train_val_AB_TSV, sep='\t')
#df.rename(columns={"text ": "text"}, inplace=True)
df = pd.read_csv(train_val_AB_TSV, delimiter=r'\t', header=None, engine='python')
df.columns =['id', 'text', 'hs', 'stereotype'] 
df = df.drop([0])
print('Number of dev sentences: {:,}\n'.format(df.shape[0]))

Already downloaded a model for the 'it' language
Reading english - 1grams ...
842
3303
Number of dev sentences: 6,839



## test set

In [86]:
#Test Tweets
df_tweets_test = pd.read_csv(test_tweets_AB_TSV, delimiter=r'\t', header=None, engine='python')

#Test News
df_news_test = pd.read_csv(test_news_AB_TSV, delimiter=r'\t', header=None, engine='python')

## reference set

In [87]:
#Reference Tweets
df_tweets_reference = pd.read_csv(reference_tweets_AB_TSV, delimiter=r'\t', header=None, engine='python')
df_tweets_reference.columns =['id', 'text', 'hs', 'stereotype'] 
print(df_tweets_reference)

#Reference News
df_news_reference = pd.read_csv(reference_news_AB_TSV, delimiter=r'\t', header=None, engine='python')
df_news_reference.columns =['id', 'text', 'hs', 'stereotype'] 
print(df_news_reference)

         id                                               text  hs  stereotype
0     11834  @user A me pare una scelta politica suicida pu...   1           0
1     12113  @user e' un perfetto musulmano!!! chi dice il ...   1           1
2     11770  Mai Tg e i giornaloni hanno parlato di questa ...   1           1
3     11937  @user Ipocriti farabutti. Fanno morire i terre...   1           1
4     11870  @user @user @user L'IMMIGRAZIONE C'E' STATO UN...   1           1
...     ...                                                ...  ..         ...
1258  10091  Il Razzismo gli viene inculcato sin da bambini...   1           1
1259  10046  PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? ...   1           1
1260  10377  FERMIAMO L'IMPOSTORE. Forse saranno necesarie ...   1           1
1261  10199  @user @user @user @user @user @user @user @use...   1           1
1262  10574  Ci vuole "coraggio" ad abbinare la parola Egua...   1           1

[1263 rows x 4 columns]
        id                 

In [11]:
# for windows
#Reference Tweets
#df_tweets_reference = pd.read_csv(reference_tweets_AB_TSV, delimiter=r'\t', header=None, engine='python')
#df_app1 = pd.DataFrame([[11834, "@user A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente in favore dell'immigrazione incontrollata...Meglio così, spariranno più velocemente!", 1, 0]], columns=['id', 'text', 'hs', 'stereotype'])
#df_tweets_reference.rename(columns={'11834': 'id', "@user A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente in favore dell'immigrazione incontrollata...Meglio così, spariranno più velocemente!": 'text', '1':'hs', '0':'stereotype'}, inplace=True)
#df_tweets_reference = df_tweets_reference.append(df_app1, ignore_index=True)

#Reference News
#df_news_reference = pd.read_csv(reference_news_AB_TSV, delimiter=r'\t', header=None, engine='python')
#df_app2 =pd.DataFrame([[11976, "Andate pure là, tanto quei fessi degli italiani.... Capito perché ci invadono? Il clandestino confessa", 1, 0]], columns=['id', 'text', 'hs', 'stereotype'])
#df_news_reference.rename(columns={'11976': 'id', "Andate pure là, tanto quei fessi degli italiani.... Capito perché ci invadono? Il clandestino confessa": 'text', '1':'hs', '0':'stereotype'}, inplace=True)
#df_news_reference = df_news_reference.append(df_app2, ignore_index=True)

In [88]:
prp = Preprocessing(italian_words=italian_dict, italian_words_gz=italian_gzip, bad_words=bad_words_dict, word_polarity_dict=word_polarity)

In [76]:
df_test = df.loc[4804:4805].copy()
df_test

Already downloaded a model for the 'it' language
Reading english - 1grams ...
842
3303


,id,text,hs,stereotype
4804,7465,@user @user @user @user @user @user @user @use...,0,0
4805,9006,@user @user @user @user @user @user @user @use...,0,0


<h2> Dev

In [77]:
prp.preprocess_df(df=df_test, column_name='text', path='../../../ReSt/dataset/haspeede2/preprocessed/dev/dev.csv')

__normalize_numbers
 Come ? ? La pacchia non doveva finire per gli immigrati ? E il reddito di cittadinanza ? E quota 100 ? 
 Come ? ? La pacchia non doveva finire per gli immigrati ? E il reddito di cittadinanza ? E quota @Dg@Dg@Dg ? 
__normalize_numbers
 E ' un rimpatrio di migranti economici 
 E ' un rimpatrio di migranti economici 


In [73]:
print(df_test["text"][4804])

 Come ? ? La pacchia non doveva finire per gli immigrati ? E il reddito di cittadinanza ? E quota @Dg@Dg@Dg ? 


<h2> Reference Twitter

In [126]:
prp.preprocess_df(df=df_tweets_reference, column_name='text', path='../../../ReSt/dataset/haspeede2/preprocessed/reference/reference_tweets.csv')

Removing URLs
Removing Tags
Feature extraction: length of the comment
Translation of emoji
Translating emoticons
Adding space before hashtag symbol '#'
Feature extraction: number of hashtags
Replacing hashtags
Normalizing hashtags
Fixing hashtags
Normalizing numbers
clean_some_punctuation
Adding space between lowercase and uppercase
Converting all emoticons written in text
Feature extraction: percentage of words written in CAPS-LOCK
Normalizing text
Feature extraction: esclamations
Feature extraction: number of questions mark
Uncensoring the bad words
Removing hashtag symbol
Removing laughs
Removing nearby equal vowels
Removing nearby equal consonants if they are more than 2
Sticking the apostrophe (text)
generate Lemma
genererate PoS
Generate Dep
Generate Word polarity
Tokenization
Sticking the apostrophe (tokens)
Generate Stemming
Replacement of the abbreviations with the respective words
Replacing Acronyms
Feature extraction: percentage of Bad Words


<h2> Reference News

In [127]:
prp.preprocess_df(df=df_news_reference, column_name='text', path='../../../ReSt/dataset/haspeede2/preprocessed/reference/reference_news.csv')

Removing URLs
Removing Tags
Feature extraction: length of the comment
Translation of emoji
Translating emoticons
Adding space before hashtag symbol '#'
Feature extraction: number of hashtags
Replacing hashtags
Normalizing hashtags
Fixing hashtags
Normalizing numbers
clean_some_punctuation
Adding space between lowercase and uppercase
Converting all emoticons written in text
Feature extraction: percentage of words written in CAPS-LOCK
Normalizing text
Feature extraction: esclamations
Feature extraction: number of questions mark
Uncensoring the bad words
Removing hashtag symbol
Removing laughs
Removing nearby equal vowels
Removing nearby equal consonants if they are more than 2
Sticking the apostrophe (text)
generate Lemma
genererate PoS
Generate Dep
Generate Word polarity
Tokenization
Sticking the apostrophe (tokens)
Generate Stemming
Replacement of the abbreviations with the respective words
Replacing Acronyms
Feature extraction: percentage of Bad Words
